In [23]:
import pandas as pd
import numpy as np
from preprocess import *
import re
import os

# import for bert stuffs
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from scipy.special import softmax

In [24]:
input_file_path = "../feature_engine/data/raw_data/juries_tiny_for_testing.csv"
df = pd.read_csv("../data/raw_data/juries_tiny_for_testing.csv")
df

,batch_num,round_num,speaker_hash,speaker_nickname,timestamp,message,majority_pct,num_flipped,flipped_pct,num_votes
0,0,0,5e7e1e0031f4e454e196c30b,niceRhino,2020-04-20T18:27:20.125Z,Hello!,1.0,1,0.333333,3
1,0,0,5e31d6e4e31c5304c46f1413,culturedCow,2020-04-20T18:27:23.764Z,Hi!,1.0,1,0.333333,3
2,0,0,5e7e4f4c31f4e454e196c9c4,spryBison,2020-04-20T18:27:27.724Z,Hello,1.0,1,0.333333,3
3,0,0,5d482ea421c9be351f762255,youngLion,2020-04-20T18:27:30.410Z,Hi,1.0,1,0.333333,3
4,0,0,5e84cc3c50f6e364321d6265,smallGiraffe,2020-04-20T18:27:35.506Z,hi,1.0,1,0.333333,3
...,...,...,...,...,...,...,...,...,...,...
92,0,2,5e7e4f4c31f4e454e196c9c4,newLion,2020-04-20T19:02:55.111Z,I say asshole under stress,0.6,0,0.000000,5
93,0,2,5d6feec65f80ae21f5c5f054,conventionalMonkey,2020-04-20T19:03:21.819Z,"Yes, she is the asshole... unfortunately. Husb...",0.6,0,0.000000,5
94,0,2,5d482ea421c9be351f762255,newPanda,2020-04-20T19:03:36.308Z,I think she is being presumptuous and acting l...,0.6,0,0.000000,5
95,0,2,5e7e4f4c31f4e454e196c9c4,newLion,2020-04-20T19:03:53.219Z,"Tha's true, she ins't considering her husband ...",0.6,0,0.000000,5


In [25]:
# preprocess 
turns = True
df = preprocess_conversation_columns(df)
df["message_lower_with_punc"] = df["message"].astype(str).apply(preprocess_text_lowercase_but_retain_punctuation)
if (turns):
    df = preprocess_naive_turns(df)

/var/folders/q6/rj1_hcnn4l1czvx3phrf8kbw0000gn/T/ipykernel_2228/4235500785.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["message_lower_with_punc"] = df["message"].astype(str).apply(preprocess_text_lowercase_but_retain_punctuation)


In [26]:
# check if data has embeddings (input, filename + turns boolean)

# ./embeddings/chats_turns vs. ./embeddings/turns
# ./sentiment_bert/chats vs ./sentiment_bert/turns
df_path = "turns" if turns else "chats"
vect_path = re.sub('../feature_engine/data/raw_data', './embeddings/' + df_path, input_file_path)
bert_path = re.sub('../feature_engine/data/raw_data', './sentiment_bert/' + df_path, input_file_path)


In [29]:
# if it does return true

if (not os.path.isfile(vect_path)):
    # generate embeddings
    print("needs vects")

if (not os.path.isfile(bert_path)):
    print("needs berts")
    # generate embeddings

needs berts


In [ ]:
def generate_bert(input_path, output_path):
    MODEL  = f"cardiffnlp/twitter-roberta-base-sentiment-latest"
    tokenizer = AutoTokenizer.from_pretrained(MODEL)
    model = AutoModelForSequenceClassification.from_pretrained(MODEL)

    # does this modify self.chat_data?
    sentiments = self.chat_data.messages.apply(get_sentiment)

    sent_arr = [list(dict.values()) for dict in sentiments]

    sent_df = pd.DataFrame(sent_arr, columns =['positive_bert', 'negative_bert', 'neutral_bert']) 
    
    output_csv_folder = '../../sentiment_bert/'

    sent_df.to_csv(output_csv_folder + dataset + '.csv')

def get_sentiment(text):

    if (pd.isnull(text)):
        return({'positive': np.nan, 'negative': np.nan, 'neutral': np.nan})
    
    encoded = tokenizer(text, return_tensors='pt')
    output = model(**encoded)

    scores = output[0][0].detach().numpy()
    scores = softmax(scores)

    # sample output format
    return({'positive': scores[2], 'negative': scores[0], 'neutral': scores[1]})

In [ ]:
def generate_vect():
    # to do, grab from ipynb